In [ ]:
import torch
import torch.nn as nn
from transformers import CLIPVisionModelWithProjection, ViTForImageClassification, AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer, LlamaForCausalLM

import sys, os, json, math
from tqdm import tqdm
import numpy as np

/home/jgryu/miniconda3/envs/nicc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def latest_version_path(cache_dir, model_name, branch = 'main'):
    model_name_dir =  "models--" + model_name.replace('/', '--')
    path = os.path.join(cache_dir, model_name_dir)
    if not os.path.isdir(os.path.join(path, 'snapshots')):
        return None
    branch_file =  os.path.join(path, 'refs', branch)
    with open(branch_file, 'r', encoding='utf-8') as file:
        revision = file.read()
    return os.path.join(path, 'snapshots', revision)

cache_directory = "../Wparam_dataset_v0/model_zoo/huggingface" 
ckpt_path = latest_version_path(cache_directory, 'meta-llama/Meta-Llama-3-8B')
net = LlamaForCausalLM.from_pretrained(ckpt_path, local_files_only=True)


ckpt_path = '/home/jgryu/Weight_compression/model_cache/models--meta-llama--Meta-Llama-3-8B/snapshots/8cde5ca8380496c9a6cc7ef3a8b46a0372a1d920'
# net = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained(ckpt_path, local_files_only=True)
state_dict = net.state_dict()

Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.63s/it]


In [ ]:
K = 16
P = 4
C = 16
U_list = []
U_inv = []
torch.manual_seed(42)
for i in range(2**K-1):
   U = torch.empty(C, P).uniform_(-1, 1)
   U_list.append(U)
   U_inv.append(torch.linalg.pinv(U))

std  = 0.011818652041256428

model.layers.0.self_attn.q_proj.weight torch.Size([4096, 4096])


/home/jgryu/miniconda3/envs/nicc/lib/python3.10/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([16, 1])) that is different to the input size (torch.Size([16])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.0002)
tensor(0.0002)
tensor(0.0002)
tensor(0.0002)
tensor(0.0002)
tensor(0.0001)
tensor(0.0002)
tensor(0.0001)
tensor(0.0003)
tensor(0.0001)
tensor(0.0002)
tensor(6.1344e-05)
tensor(0.0003)
tensor(0.0002)
tensor(0.0002)
tensor(0.0002)
tensor(0.0002)
tensor(0.0003)
tensor(0.0001)
tensor(0.0002)
tensor(0.0003)
tensor(0.0002)
tensor(7.1355e-05)
tensor(0.0001)
tensor(5.0768e-05)
tensor(0.0002)
tensor(0.0002)
tensor(0.0001)
tensor(0.0002)
tensor(0.0002)
tensor(0.0002)
tensor(0.0002)
tensor(8.2367e-05)
tensor(0.0001)
tensor(0.0001)
tensor(0.0003)
tensor(6.8107e-05)
tensor(0.0002)
tensor(0.0001)
tensor(0.0002)
tensor(5.5311e-05)
tensor(0.0002)
tensor(0.0002)
tensor(0.0001)
tensor(0.0002)
tensor(0.0002)
tensor(0.0002)
tensor(0.0002)
tensor(0.0002)
tensor(8.2218e-05)
tensor(0.0001)
tensor(7.5897e-05)
tensor(0.0001)
tensor(0.0002)
tensor(0.0002)
tensor(0.0004)
tensor(9.5540e-05)
tensor(9.0581e-05)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(7.6210e-05)
tensor(5.71

KeyboardInterrupt: 

In [11]:
mse_fn = nn.MSELoss()
total_mse = 0
count = 0
for k, v in state_dict.items():
   if 'mlp' not in k: continue
   print(k, v.shape)
   v_reshaped = v.view(-1, C)
   for w in v_reshaped:
      min_mse = 7000
      for i in range(2**K-1):
         t = torch.mm(U_inv[i], w.reshape(-1, 1))
         mse = mse_fn(w, torch.mm(U_list[i],t))
         if mse < min_mse:
            min_mse = mse
      print(min_mse / std**2)
      total_mse += min_mse
      count += 1

print('MSE: ', total_mse/count / std**2)

model.layers.0.mlp.gate_proj.weight torch.Size([14336, 4096])


/home/jgryu/miniconda3/envs/nicc/lib/python3.10/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([16, 1])) that is different to the input size (torch.Size([16])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.7754)
tensor(0.8012)
tensor(0.8179)
tensor(0.6085)
tensor(0.9881)
tensor(0.7533)
tensor(0.9627)
tensor(0.5357)
tensor(1.0474)
tensor(0.3673)
tensor(0.8881)
tensor(0.8982)
tensor(1.2313)
tensor(0.6349)
tensor(0.4406)
tensor(1.0828)
tensor(0.5781)
tensor(0.4664)
tensor(0.4118)
tensor(1.2403)
tensor(1.1039)
tensor(2.2953)
tensor(0.7458)
tensor(0.7019)
tensor(0.7425)
tensor(1.0089)
tensor(0.4489)
tensor(0.7486)
tensor(0.5231)
tensor(0.6807)
tensor(0.7777)
tensor(0.5776)
tensor(1.0870)
tensor(0.9347)
tensor(0.8875)
tensor(0.6165)
tensor(1.0875)
tensor(0.6952)
tensor(1.0920)
tensor(0.8387)
tensor(0.9150)
tensor(0.6410)
tensor(0.7403)
tensor(0.9867)
tensor(1.0832)
tensor(1.0811)
tensor(0.6985)
tensor(0.8086)
tensor(0.7048)
tensor(1.0493)
tensor(0.9274)
tensor(0.5901)
tensor(1.1346)
tensor(0.7944)
tensor(0.8324)
tensor(1.1316)
tensor(0.9780)
tensor(1.7643)
tensor(0.2556)
tensor(1.1892)
tensor(0.5293)
tensor(0.9238)
tensor(0.2860)
tensor(0.8288)
tensor(1.1586)
tensor(1.2599)
tensor(0.7

KeyboardInterrupt: 

In [9]:
std  = 0.011818652041256428